In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.chdir('/content/gdrive/My Drive/FYP/Speaker/Classification')
os.environ['TF_CPP_MIN_LOG_LEVEL']= '3'

import tensorflow as tf
tf.autograph.set_verbosity(0)

In [3]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from model import createModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

In [4]:
def getData(x, s, le):
    X, y = [], []
    speakers = list(s.keys())

    for i in range(len(speakers)):
        l, h = s[speakers[i]]
        for j in range(0, h-l+1):
            X_, y_ = abs(x[i][j]), le.transform([speakers[i]])
            X_ = np.reshape(X_, (X_.shape[0], X_.shape[1], 1))
            # y_ = to_categorical(y_, num_classes=100).reshape(-1)
            X.append(X_)
            y.append(y_)

    return np.array(X), np.array(y)

In [5]:
DATA_PATH = '../Data/Data/'
with open(DATA_PATH + 'train.pickle', 'rb') as f:
    x, s = pickle.load(f)

In [6]:
speakers = sorted(list(s.keys()))
le = LabelEncoder()
le.fit(speakers)

LabelEncoder()

In [7]:
X, y = getData(x, s, le)
del x, s

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, shuffle=True)
del X, y

In [9]:
EPOCHS = 20
BATCH_SIZE = 256

In [10]:
model = createModel(input_shape=(300, 128, 1), name='conv_2d')
model.summary()

Model: "conv_2d"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 128, 1)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 2016)              27912     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2065408   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
output (Dense)               (None, 100)               2570

In [11]:
checkpoint = ModelCheckpoint('Model/Conv_2D/conv2d_{epoch:02d}.h5', save_weights_only=True, period=10)
history = model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val), callbacks=[checkpoint])

Epoch 1/20
55/55 [==============================] - 18s 177ms/step - loss: 4.6480 - accuracy: 0.0123 - val_loss: 4.6033 - val_accuracy: 0.0121
Epoch 2/20
55/55 [==============================] - 9s 160ms/step - loss: 4.5936 - accuracy: 0.0124 - val_loss: 4.5508 - val_accuracy: 0.0183
Epoch 3/20
55/55 [==============================] - 9s 159ms/step - loss: 4.3455 - accuracy: 0.0227 - val_loss: 3.8990 - val_accuracy: 0.0552
Epoch 4/20
55/55 [==============================] - 9s 159ms/step - loss: 3.8315 - accuracy: 0.0617 - val_loss: 3.4344 - val_accuracy: 0.1207
Epoch 5/20
55/55 [==============================] - 9s 159ms/step - loss: 3.3387 - accuracy: 0.1346 - val_loss: 2.8960 - val_accuracy: 0.2204
Epoch 6/20
55/55 [==============================] - 9s 159ms/step - loss: 2.8741 - accuracy: 0.2355 - val_loss: 2.3810 - val_accuracy: 0.3426
Epoch 7/20
55/55 [==============================] - 9s 159ms/step - loss: 2.2415 - accuracy: 0.3725 - val_loss: 2.1572 - val_accuracy: 0.3943
Epoch

In [12]:
with open('Model/Conv_2D/conv2d_history.pickle', 'wb') as f:
    pickle.dump(history.history, f)

In [13]:
model.get_layer('sequential').save_weights('Model/Conv_2D/conv2d_core.h5')

In [14]:
K.clear_session()

In [15]:
model = createModel(input_shape=(300, 128, 1), name='lstm')
model.summary()

Model: "lstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 128, 1)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 9600)              37248     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              9831424   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
output (Dense)               (None, 100)               25700  

In [16]:
checkpoint = ModelCheckpoint('Model/LSTM/lstm_{epoch:02d}.h5', save_weights_only=True, period=10)
history = model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val), callbacks=[checkpoint])

Epoch 1/20
55/55 [==============================] - 9s 108ms/step - loss: 4.5779 - accuracy: 0.0172 - val_loss: 4.0284 - val_accuracy: 0.0537
Epoch 2/20
55/55 [==============================] - 5s 95ms/step - loss: 3.8076 - accuracy: 0.0862 - val_loss: 3.1144 - val_accuracy: 0.1774
Epoch 3/20
55/55 [==============================] - 5s 94ms/step - loss: 2.8247 - accuracy: 0.2287 - val_loss: 2.2685 - val_accuracy: 0.3671
Epoch 4/20
55/55 [==============================] - 5s 94ms/step - loss: 1.8799 - accuracy: 0.4571 - val_loss: 1.2692 - val_accuracy: 0.6220
Epoch 5/20
55/55 [==============================] - 5s 95ms/step - loss: 1.2775 - accuracy: 0.6135 - val_loss: 1.1295 - val_accuracy: 0.6517
Epoch 6/20
55/55 [==============================] - 5s 95ms/step - loss: 0.8936 - accuracy: 0.7173 - val_loss: 0.7005 - val_accuracy: 0.7884
Epoch 7/20
55/55 [==============================] - 5s 94ms/step - loss: 0.6396 - accuracy: 0.7934 - val_loss: 0.7726 - val_accuracy: 0.7615
Epoch 8/20
5

In [17]:
with open('Model/LSTM/lstm_history.pickle', 'wb') as f:
    pickle.dump(history.history, f)

In [18]:
model.get_layer('sequential').save_weights('Model/LSTM/lstm_core.h5')

In [19]:
K.clear_session()

In [10]:
model = createModel(input_shape=(300, 128, 1), name='conv_1d')
model.summary()

Model: "conv_1d"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 128, 1)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 128)               43800     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              132096    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
output (Dense)               (None, 100)               2570

In [11]:
checkpoint = ModelCheckpoint('Model/Conv_1D/conv1d_{epoch:02d}.h5', save_weights_only=True, period=10)
history = model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val), callbacks=[checkpoint])

Epoch 1/20
55/55 [==============================] - 113s 2s/step - loss: 4.6059 - accuracy: 0.0099 - val_loss: 4.2551 - val_accuracy: 0.0193
Epoch 2/20
55/55 [==============================] - 101s 2s/step - loss: 4.1853 - accuracy: 0.0251 - val_loss: 3.9820 - val_accuracy: 0.0582
Epoch 3/20
55/55 [==============================] - 101s 2s/step - loss: 3.8881 - accuracy: 0.0541 - val_loss: 3.5814 - val_accuracy: 0.0849
Epoch 4/20
55/55 [==============================] - 101s 2s/step - loss: 3.5631 - accuracy: 0.0913 - val_loss: 3.3081 - val_accuracy: 0.1341
Epoch 5/20
55/55 [==============================] - 101s 2s/step - loss: 3.2568 - accuracy: 0.1454 - val_loss: 2.9167 - val_accuracy: 0.2012
Epoch 6/20
55/55 [==============================] - 101s 2s/step - loss: 2.9178 - accuracy: 0.2023 - val_loss: 2.7189 - val_accuracy: 0.2575
Epoch 7/20
55/55 [==============================] - 101s 2s/step - loss: 2.6177 - accuracy: 0.2736 - val_loss: 2.4089 - val_accuracy: 0.3199
Epoch 8/20
55

In [12]:
with open('Model/Conv_1D/conv1d_history.pickle', 'wb') as f:
    pickle.dump(history.history, f)

In [13]:
model.get_layer('sequential').save_weights('Model/Conv_1D/conv1d_core.h5')